In [1]:
import sys
sys.path.append('../SleepClasses/')  # Aggiungi il percorso della cartella2

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures, StandardScaler
from AggregatedData import AggregatedData
import plotly.express as px
import plotly.graph_objects as go
from datetime import timedelta
from sklearn.svm import OneClassSVM
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
import shap

# Mostra tutte le colonne
pd.set_option('display.max_columns', None)

/Users/boe/Desktop/Unimi/Tesi/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
sleep_dfs_agg = pd.read_pickle(r'../SleepData/sleep_dfs_agg.pkl')
sleep_dfs_gran = pd.read_pickle(r'../SleepData/sleep_dfs_gran.pkl')
agg = AggregatedData(sleep_dfs_gran['2'], sleep_dfs_agg['2'])

df = agg.getAggregatedDataDf()

/Users/boe/Desktop/Unimi/Tesi/code/anomaly/../SleepClasses/AggregatedData.py:984: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat(merged_aggregated.values(), ignore_index=True)
/Users/boe/Desktop/Unimi/Tesi/code/anomaly/../SleepClasses/AggregatedData.py:1041: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  modified_rows_df = pd.concat([modified_rows_df, df.loc[[idx]]])
/Users/boe/Desktop/Unimi/Tesi/code/anomaly/../SleepClasses/AggregatedData.py:1044: FutureWarning: The behavi

In [3]:
# Assicurati che la colonna 'Date' sia in formato datetime
df['Date'] = pd.to_datetime(df['Date'])

# Crea la colonna 'has_evening_nap' inizializzata a False
df['HasEveningNap'] = False

# Rimuovi le righe dove 'Nap' è True e aggiorna la data del giorno successivo
to_remove = df[df['Nap'] == True]
for idx, row in to_remove.iterrows():
    # Trova la riga del giorno successivo
    next_day = row['Date'] + timedelta(days=1)
    
    # Verifica se esiste una riga per il giorno successivo e se 'Nap' è False
    next_day_row = df[(df['Date'] == next_day) & (df['Nap'] == False)]
    
    # Se la riga esiste, imposta 'has_evening_nap' a True
    if not next_day_row.empty:
        df.loc[next_day_row.index, 'HasEveningNap'] = True

# Rimuovi le righe con 'Nap' True
df = df[df['Nap'] == False].reset_index(drop=True)

In [4]:
df

,Date,StartValue,EndValue,LightSleepDuration,DeepSleepDuration,RemSleepDuration,WakeUpDuration,TotalSleepTime,NumRemEpisodes,SleepEfficiency,SleepLatency,TotalTimeInBed,ApneaHypopneaIndex,BreathingDisturbancesIntensity,AsleepDuration,MVTActiveDuration,NightEvents,Snoring,WakeupLatency,Waso,SnoringEpisodeCount,SleepScore,SleepCount,HashDeviceID,Model,ModelID,ManualSleepDuration,UndefinedSleepDuration,Nap,HrAverage,HrMax,HrMin,RmssdAverage,RmssdMax,RmssdMin,RrAverage,RrMax,RrMin,Sdnn_1Average,Sdnn_1Max,Sdnn_1Min,SnoringAverage,SnoringMax,SnoringMin,MVTScoreAverage,MVTScoreMax,MVTScoreMin,WakeUpCount,OutOfBedCount,OutOfBedTime,OutOfBed,hashDeviceid,HasEveningNap
0,2023-07-25,2023-07-25 00:08:00,2023-07-25 09:45:00,14520,5880,4980,8400,25380.0,4,0.75,4320,33780,59.00,117.0,0,0.0,"{""1"":[0,14460,16200],""2"":[4320,10680,3420,5460...",120.0,2100,2820,1.0,69.0,2,NaN,32,63,0,0,False,62,76,49,0,0,0,17.0,24.0,9.0,0,0,0,0,0,0,0.0,0.0,0.0,4,3,0,None,0,True
1,2023-07-26,2023-07-25 23:27:00,2023-07-26 07:32:00,21240,3600,1080,3180,25920.0,1,0.89,2460,29100,30.41,60.0,0,0.0,"{""1"":[0],""2"":[2460,8880,17160],""3"":[10980,1716...",0.0,0,720,0.0,74.0,1,NaN,32,63,0,0,False,62,82,50,0,0,0,16.0,22.0,11.0,0,0,0,0,0,0,0.0,0.0,0.0,2,0,0,None,0,True
2,2023-07-27,2023-07-26 23:54:00,2023-07-27 09:33:00,15480,6000,4680,8160,26160.0,3,0.76,4320,34320,41.18,82.0,0,0.0,"{""1"":[0,27240,6540],""2"":[4320,23280,6240],""3"":...",1320.0,0,4260,3.0,72.0,2,NaN,32,63,0,0,False,61,74,48,0,0,0,16.0,26.0,9.0,0,0,0,0,0,0,0.0,0.0,0.0,2,2,0,None,0,False
3,2023-07-28,2023-07-28 02:33:00,2023-07-28 07:44:00,7500,3300,1680,5700,12480.0,2,0.69,900,18180,-1.00,-1.0,0,0.0,"{""1"":[0,5580,5280],""2"":[900,5280,5820],""3"":[52...",0.0,0,5280,0.0,20.0,1,NaN,32,63,0,0,False,62,76,50,0,0,0,17.0,22.0,12.0,0,0,0,0,0,0,0.0,0.0,0.0,2,2,0,None,0,True
4,2023-08-04,2023-08-03 23:11:00,2023-08-04 08:14:00,15420,10380,0,2880,25800.0,0,0.90,720,28680,32.00,64.0,0,0.0,NaN,360.0,0,6060,1.0,73.0,1,NaN,16,93,0,0,False,60,81,48,0,0,0,16.0,23.0,10.0,0,0,0,0,0,0,0.0,0.0,0.0,4,0,0,None,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
429,2024-11-22,2024-11-22 00:41:00,2024-11-22 08:30:00,21120,2100,900,3600,24120.0,1,0.87,1620,27720,65.14,130.0,0,300.0,"{""1"":[0,19440],""2"":[1620,900,12900,4620,3300],...",840.0,0,2400,3.0,67.0,2,NaN,32,63,0,0,False,61,80,48,52,99,0,15.0,27.0,9.0,66,202,0,3,100,0,3.0,100.0,0.0,4,1,840,"[{'from_value': 1, 'to_value': 0, 'next_value'...",0,True
430,2024-11-23,2024-11-22 23:33:00,2024-11-23 09:45:00,24000,2940,4920,4320,31860.0,3,0.88,1380,36180,53.92,107.0,0,720.0,"{""1"":[0,16260,5100],""2"":[1380,2700,12180,6000]...",3900.0,0,3480,13.0,80.0,2,NaN,32,63,0,0,False,59,76,48,55,95,0,15.0,23.0,9.0,74,201,0,10,100,0,10.0,100.0,0.0,2,1,420,"[{'from_value': 1, 'to_value': 0, 'next_value'...",0,True
431,2024-11-24,2024-11-23 23:26:00,2024-11-24 08:41:00,25080,600,5940,900,31620.0,5,0.97,900,32520,63.60,127.0,0,900.0,"{""1"":[0,10740,16680],""2"":[900,9840,16680],""3"":...",6660.0,0,780,16.0,91.0,2,NaN,32,63,0,0,False,72,93,54,48,94,0,18.0,25.0,10.0,61,241,0,20,100,0,20.0,100.0,0.0,0,0,0,[],0,True
432,2024-11-25,2024-11-24 23:55:00,2024-11-25 08:17:00,18480,2220,7140,1920,27840.0,4,0.94,720,29760,44.42,88.0,0,540.0,"{""1"":[0,26220],""2"":[720,4260,18000,3300],""3"":[...",1320.0,0,1560,4.0,82.0,2,NaN,32,63,0,0,False,58,78,47,50,96,0,15.0,20.0,9.0,62,196,0,4,100,0,4.0,100.0,0.0,3,2,720,"[{'from_value': 2, 'to_value': 0, 'next_value'...",0,True


In [5]:
# Funzione per riempire i valori nulli e tracciare le righe modificate
def fill_zeros_columns(df, columns_to_check):
    modified_rows_df = pd.DataFrame(columns=df.columns)
    # Itera attraverso ogni riga del DataFrame
    for idx, row in df.iterrows():
        modified = False
        previous_rows = df[(df.index < idx)].tail(7)
        # Per ogni colonna specificata, calcola e riempi i valori nulli
        for col in columns_to_check:
            if(col in df.columns):            
                if row[col] == 0:
                    if not previous_rows.empty:
                        # Calcola la media delle 7 righe precedenti per la colonna specificata e aggiorna la cella corrente
                        df.at[idx, col] = previous_rows[col].mean()
                    else:
                        # Se non ci sono valori precedenti, riempi con 0 come valore predefinito
                        df.at[idx, col] = 0.0
                    #inserisco nella lista delle righe modificate
                    modified = True
            
        if(modified):
            modified_rows_df = pd.concat([modified_rows_df, df.loc[[idx]]])
    return df, modified_rows_df

In [6]:
def filter_dataframe(df, threshold=0.5):
    df = df.drop(columns=['AsleepDuration', 'NightEvents', 'HashDeviceID', 'Model', 'ModelID', 'ManualSleepDuration', 'UndefinedSleepDuration', 'Nap', 'OutOfBed', 'hashDeviceid'])

    # Calcola la percentuale di zeri in ogni colonna
    zero_percentage = (df == 0).sum() / len(df)

    # Droppa le colonne che superano la soglia
    df_filtered = df.loc[:, zero_percentage <= threshold]


    # Specifica le colonne da rimuovere
    columns_to_check = ['SleepEfficiency', 'SleepLatency', 'ApneaHypopneaIndex', 'BreathingDisturbancesIntensity', 'WakeupLatency', 'Waso', 'SleepScore', 'HrAverage', 'HrMin',
                        'RmssdAverage', 'RmssdMax', 'RmssdMin', 'RrAverage', 'RrMax', 'RrMin', 'Sdnn_1Average', 'Sdnn_1Max', 'Sdnn_1Min','MVTScoreAverage', 'MVTScoreMax', 'MVTScoreMin']

    df_filtered, modified_rows_df = fill_zeros_columns(df_filtered, columns_to_check)
    
    return df_filtered, modified_rows_df
df_filtered, modified_rows_df = filter_dataframe(df)

/var/folders/tf/6669z1mn2pv1y3dv4rdzn7200000gn/T/ipykernel_15521/1184021017.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  modified_rows_df = pd.concat([modified_rows_df, df.loc[[idx]]])
/var/folders/tf/6669z1mn2pv1y3dv4rdzn7200000gn/T/ipykernel_15521/1184021017.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3754.285714285714' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[idx, col] = previous_rows[col].mean()


In [7]:
df_filtered


,Date,StartValue,EndValue,LightSleepDuration,DeepSleepDuration,RemSleepDuration,WakeUpDuration,TotalSleepTime,NumRemEpisodes,SleepEfficiency,SleepLatency,TotalTimeInBed,ApneaHypopneaIndex,BreathingDisturbancesIntensity,Snoring,Waso,SnoringEpisodeCount,SleepScore,SleepCount,HrAverage,HrMax,HrMin,RmssdAverage,RmssdMax,RrAverage,RrMax,RrMin,Sdnn_1Average,Sdnn_1Max,SnoringAverage,SnoringMax,MVTScoreAverage,MVTScoreMax,WakeUpCount,OutOfBedCount,OutOfBedTime,HasEveningNap
0,2023-07-25,2023-07-25 00:08:00,2023-07-25 09:45:00,14520,5880,4980,8400,25380.0,4,0.75,4320,33780,59.00,117.0,120.0,2820.0,1.0,69.0,2,62,76,49,0.0,0.0,17.0,24.0,9.0,0.0,0.0,0,0,0.0,0.0,4,3,0,True
1,2023-07-26,2023-07-25 23:27:00,2023-07-26 07:32:00,21240,3600,1080,3180,25920.0,1,0.89,2460,29100,30.41,60.0,0.0,720.0,0.0,74.0,1,62,82,50,0.0,0.0,16.0,22.0,11.0,0.0,0.0,0,0,0.0,0.0,2,0,0,True
2,2023-07-27,2023-07-26 23:54:00,2023-07-27 09:33:00,15480,6000,4680,8160,26160.0,3,0.76,4320,34320,41.18,82.0,1320.0,4260.0,3.0,72.0,2,61,74,48,0.0,0.0,16.0,26.0,9.0,0.0,0.0,0,0,0.0,0.0,2,2,0,False
3,2023-07-28,2023-07-28 02:33:00,2023-07-28 07:44:00,7500,3300,1680,5700,12480.0,2,0.69,900,18180,-1.00,-1.0,0.0,5280.0,0.0,20.0,1,62,76,50,0.0,0.0,17.0,22.0,12.0,0.0,0.0,0,0,0.0,0.0,2,2,0,True
4,2023-08-04,2023-08-03 23:11:00,2023-08-04 08:14:00,15420,10380,0,2880,25800.0,0,0.90,720,28680,32.00,64.0,360.0,6060.0,1.0,73.0,1,60,81,48,0.0,0.0,16.0,23.0,10.0,0.0,0.0,0,0,0.0,0.0,4,0,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
429,2024-11-22,2024-11-22 00:41:00,2024-11-22 08:30:00,21120,2100,900,3600,24120.0,1,0.87,1620,27720,65.14,130.0,840.0,2400.0,3.0,67.0,2,61,80,48,52,99,15.0,27.0,9.0,66,202,3,100,3.0,100.0,4,1,840,True
430,2024-11-23,2024-11-22 23:33:00,2024-11-23 09:45:00,24000,2940,4920,4320,31860.0,3,0.88,1380,36180,53.92,107.0,3900.0,3480.0,13.0,80.0,2,59,76,48,55,95,15.0,23.0,9.0,74,201,10,100,10.0,100.0,2,1,420,True
431,2024-11-24,2024-11-23 23:26:00,2024-11-24 08:41:00,25080,600,5940,900,31620.0,5,0.97,900,32520,63.60,127.0,6660.0,780.0,16.0,91.0,2,72,93,54,48,94,18.0,25.0,10.0,61,241,20,100,20.0,100.0,0,0,0,True
432,2024-11-25,2024-11-24 23:55:00,2024-11-25 08:17:00,18480,2220,7140,1920,27840.0,4,0.94,720,29760,44.42,88.0,1320.0,1560.0,4.0,82.0,2,58,78,47,50,96,15.0,20.0,9.0,62,196,4,100,4.0,100.0,3,2,720,True


In [8]:
def aggrega_dati_settimanali(df, days = 7, min_records = 4, overlap_rows=1):
    grouped_data = []
    n = len(df)
    
    # Inizializza l'indice iniziale per la finestra corrente
    start_idx = 0

    while start_idx < n:
        # Calcoliamo la data di inizio e fine della finestra
        start_date = df['Date'].iloc[start_idx]
        end_date = start_date + timedelta(days=days)
        
        # Troviamo l'intervallo di righe che rientra nella finestra
        end_idx = start_idx
        while end_idx < n and df['Date'].iloc[end_idx] <= end_date:
            end_idx += 1

        # Verifica se il gruppo contiene abbastanza righe
        if (end_idx - start_idx) >= min_records:
            grouped_data.append(df.iloc[start_idx:end_idx])
        
        # Aggiorna l'indice di inizio per la finestra successiva
        # Manteniamo solo le ultime overlap_rows righe
        start_idx = max(end_idx - overlap_rows, start_idx + 1)
    
    all_stats = []
    for i, df in enumerate(grouped_data):
        # Seleziona solo le colonne numeriche
        columnsNotToAggregate = ['Date', 'StartValue', 'EndValue', 'HasEveningNap']
        columns_to_check = [col for col in df.columns if col not in columnsNotToAggregate]
        numeric_data = df[columns_to_check]
        
        # Calcola le statistiche
        stats = numeric_data.agg(['mean', 'std']).T
        stats_flattened = stats.stack()  # Trasforma in formato lungo
        stats_flattened.index = [f"{col}_{stat}" for col, stat in stats_flattened.index]  # Combina nomi
        stats_flattened = stats_flattened.to_dict()  # Converte in dizionario

        # Calcola la somma di 'True' nella colonna 'has_evening_nap'
        has_evening_nap_count = df['HasEveningNap'].sum() if 'HasEveningNap' in df else 0

        # Calcola il primo e ultimo valore della colonna 'Date'
        first_date = df['Date'].min() if 'Date' in df else None
        last_date = df['Date'].max() if 'Date' in df else None

        # Aggiungi statistiche extra al dizionario
        stats_flattened.update({
            'EveningNapCount': has_evening_nap_count,
            'FirstDate': first_date,
            'LastDate': last_date
        })
        all_stats.append(stats_flattened)

    # Converte in DataFrame
    combined_stats = pd.DataFrame(all_stats)
    # Riordina le colonne per avere FirstDate, LastDate e Group come prime
    ordered_columns = ['FirstDate', 'LastDate'] + [col for col in combined_stats.columns if col not in ['FirstDate', 'LastDate']]
    combined_stats = combined_stats[ordered_columns]
    # Crea un'istanza dello scaler
    scaler = MinMaxScaler()
    
    # Seleziona le colonne da scalare
    columns_to_scale = [col for col in combined_stats.columns if col not in ['FirstDate', 'LastDate']]
    
    # Applica lo scaling ai dati numerici
    df_scaled = combined_stats.copy()
    df_scaled[columns_to_scale] = scaler.fit_transform(combined_stats[columns_to_scale])
    
    # Converte i dati scalati in float64
    df_scaled[columns_to_scale] = df_scaled[columns_to_scale].astype('float64')
    
    return df_scaled
df_aggregato = aggrega_dati_settimanali(df_filtered, days=7, min_records=4, overlap_rows=6)

In [9]:
df_aggregato

,FirstDate,LastDate,LightSleepDuration_mean,LightSleepDuration_std,DeepSleepDuration_mean,DeepSleepDuration_std,RemSleepDuration_mean,RemSleepDuration_std,WakeUpDuration_mean,WakeUpDuration_std,TotalSleepTime_mean,TotalSleepTime_std,NumRemEpisodes_mean,NumRemEpisodes_std,SleepEfficiency_mean,SleepEfficiency_std,SleepLatency_mean,SleepLatency_std,TotalTimeInBed_mean,TotalTimeInBed_std,ApneaHypopneaIndex_mean,ApneaHypopneaIndex_std,BreathingDisturbancesIntensity_mean,BreathingDisturbancesIntensity_std,Snoring_mean,Snoring_std,Waso_mean,Waso_std,SnoringEpisodeCount_mean,SnoringEpisodeCount_std,SleepScore_mean,SleepScore_std,SleepCount_mean,SleepCount_std,HrAverage_mean,HrAverage_std,HrMax_mean,HrMax_std,HrMin_mean,HrMin_std,RmssdAverage_mean,RmssdAverage_std,RmssdMax_mean,RmssdMax_std,RrAverage_mean,RrAverage_std,RrMax_mean,RrMax_std,RrMin_mean,RrMin_std,Sdnn_1Average_mean,Sdnn_1Average_std,Sdnn_1Max_mean,Sdnn_1Max_std,SnoringAverage_mean,SnoringAverage_std,SnoringMax_mean,SnoringMax_std,MVTScoreAverage_mean,MVTScoreAverage_std,MVTScoreMax_mean,MVTScoreMax_std,WakeUpCount_mean,WakeUpCount_std,OutOfBedCount_mean,OutOfBedCount_std,OutOfBedTime_mean,OutOfBedTime_std,EveningNapCount
0,2023-07-25,2023-07-28,0.000000,0.535859,0.704678,0.160979,0.257345,0.397892,0.576271,0.263234,0.203902,0.562029,0.185780,0.282791,0.168378,0.260136,0.805333,0.917877,0.318807,0.594236,0.139840,0.744788,0.139256,0.731952,0.066390,0.194480,0.253430,0.157987,0.070175,0.189588,0.409408,1.000000,0.163636,0.287678,0.315175,0.000000,0.000000,0.196622,0.337017,0.036916,0.000000,0.000000,0.000000,0.000000,0.52,0.348155,0.218750,0.503304,0.653631,0.452267,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.5500,0.228271,0.586207,0.657129,0.000000,0.000000,0.2
1,2023-09-01,2023-09-08,0.441799,0.260183,0.519006,0.137947,0.272543,0.365415,0.394539,0.293147,0.448710,0.331610,0.263188,0.402687,0.494867,0.290037,0.760000,0.537404,0.437804,0.326890,0.276158,0.297131,0.274310,0.295782,0.114799,0.124297,0.274481,0.140930,0.184211,0.174608,0.537456,0.841131,0.218182,0.257881,0.233463,0.144101,0.145474,0.295334,0.295580,0.057826,0.693431,0.540506,0.515957,0.198199,0.12,0.279145,0.593750,0.220749,0.251397,0.351250,0.749526,0.308156,0.273343,0.160044,0.095588,0.120969,1.00000,0.000000,0.109244,0.110311,1.000000,0.000000,0.2875,0.356825,0.327586,0.435815,0.162246,0.106901,0.4
2,2023-09-03,2023-09-10,0.429453,0.278238,0.562865,0.149690,0.235056,0.379016,0.394539,0.239083,0.435494,0.335127,0.159977,0.387630,0.488706,0.235984,0.890667,0.394632,0.425054,0.337218,0.247348,0.301091,0.245498,0.300239,0.084371,0.106316,0.171518,0.180157,0.149123,0.167094,0.537456,0.747604,0.218182,0.257881,0.291829,0.125151,0.232759,0.156564,0.350829,0.033927,0.695864,0.536139,0.509309,0.194155,0.12,0.279145,0.531250,0.252792,0.150838,0.161165,0.722960,0.346469,0.263178,0.171153,0.058824,0.085987,0.87500,0.612372,0.083884,0.070194,1.000000,0.000000,0.3250,0.298383,0.327586,0.435815,0.138846,0.105676,0.4
3,2023-09-05,2023-09-12,0.528219,0.257266,0.565789,0.138655,0.327254,0.326396,0.378531,0.139089,0.564506,0.212298,0.263188,0.372146,0.556468,0.074420,0.853333,0.417764,0.539193,0.293704,0.233786,0.219708,0.231993,0.220739,0.082988,0.108102,0.171106,0.254971,0.140351,0.175524,0.681185,0.255650,0.109091,0.257881,0.385214,0.174240,0.368534,0.180254,0.350829,0.095547,0.693431,0.573308,0.521277,0.169473,0.12,0.279145,0.531250,0.252792,0.100559,0.193230,0.717268,0.340343,0.259789,0.176423,0.051471,0.092043,0.87500,0.612372,0.087464,0.068134,1.000000,0.000000,0.4750,0.548115,0.431034,0.478431,0.299532,0.447112,0.3
4,2023-09-07,2023-09-14,0.568783,0.256463,0.678363,0.255086,0.323202,0.312090,0.355932,0.133772,0.639396,0.277458,0.237385,0.350085,0.599589,0.055967,0.808000,0.334267,0.596870,0.356543,0.168004,0.135511,0.166267,0.133995,0.078838,0.134669,0.172720,0.262797,0.131579,0.231505,0.717770,0.295425,0.163636,0.266338,0.455253,0.182377,0.349138,0.1928

In [10]:
# Selezioniamo i dati che si trovano nel primo mese (ad esempio, un mese dopo la 'FirstDate')
train_start_date = "2023-09-01"  # Puoi scegliere la data minima o la data che desideri come inizio
train_end_date = "2023-10-01"  # Aggiungi 60 giorni per coprire due mesi

test_start_date = "2023-10-01"  # Puoi scegliere la data minima o la data che desideri come inizio
test_end_date = "2024-05-01"  # Aggiungi 60 giorni per coprire due mesi

# Filtriamo il dataframe per includere solo le righe con 'FirstDate' dentro questo periodo di un mese
train_data = df_aggregato[(df_aggregato['FirstDate'] >= train_start_date) & (df_aggregato['FirstDate'] <= train_end_date)]

# Ora possiamo estrarre i dati che si trovano oltre il mese di addestramento
test_data = df_aggregato[(df_aggregato['FirstDate'] >= test_start_date) & (df_aggregato['FirstDate'] <= test_end_date)]
# Seleziona le colonne da scalare
columns_for_train_model = [col for col in train_data.columns if col not in ['FirstDate', 'LastDate', 'Group']]

Rifai tutte le anomalie non per colonna ma per tutte le colonne insieme

In [11]:
# Inizializza e allena il modello One-Class SVM
model = OneClassSVM(kernel='poly', nu=0.005, gamma='scale')
model.fit(train_data[columns_for_train_model])

# Predizioni
train_data["anomaly"] = model.predict(train_data[columns_for_train_model])
test_data["anomaly"] = model.predict(test_data[columns_for_train_model])

# Unisci training e test per la visualizzazione
train_data["set"] = "train"
test_data["set"] = "test"

df_combined = pd.concat([train_data, test_data])

# Visualizzazione per ogni feature
for feature in columns_for_train_model:
    fig = go.Figure()

    # Parte di training
    fig.add_trace(go.Scatter(
        x=train_data["FirstDate"],
        y=train_data[feature],
        mode="lines",
        name=f"{feature} (Train)",
        line=dict(color="green", width=2)
    ))

    # Parte di test
    fig.add_trace(go.Scatter(
        x=test_data["FirstDate"],
        y=test_data[feature],
        mode="lines",
        name=f"{feature} (Test)",
        line=dict(color="blue", width=2)
    ))

    # Anomalie
    anomalies = df_combined[df_combined["anomaly"] == -1]
    fig.add_trace(go.Scatter(
        x=anomalies["FirstDate"],
        y=anomalies[feature],
        mode="markers",
        name="Anomalies",
        marker=dict(color="red", size=8, symbol="x")
    ))

    # Layout del grafico
    fig.update_layout(
        title=f"Anomaly Detection for {feature}",
        xaxis_title="Date",
        yaxis_title=feature,
        template="plotly_white",
        showlegend=True,
        height=600,
        width=1000
    )

    # Mostra il grafico
    fig.show()

/var/folders/tf/6669z1mn2pv1y3dv4rdzn7200000gn/T/ipykernel_15521/1819693553.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data["anomaly"] = model.predict(train_data[columns_for_train_model])
/var/folders/tf/6669z1mn2pv1y3dv4rdzn7200000gn/T/ipykernel_15521/1819693553.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data["anomaly"] = model.predict(test_data[columns_for_train_model])
/var/folders/tf/6669z1mn2pv1y3dv4rdzn7200000gn/T/ipykernel_15521/1819693553.py:10: SettingWithCopyWarning: 


One Class SVM non va bene perchè non prende come ground truth tutto il dataset ma rileva delle anomalie basandosi sul valore di mu, questo a noi non va bene, secondo la ground truth (i valori del primo mese) deve stimare i dati anomali